# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

C:\Users\Niek\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Niek\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]
corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [ ]:
#create co-occurrence matrix
shape = (V, V)
matrix = np.zeros(shape)
np.fill_diagonal(matrix, 0)
for sentence in corpus:
    for i in range(0, len(sentence)):
        coword = sentence[i+1:i+window_size_corpus+1]
        for j in coword:
            matrix[sentence[i], j] = matrix[sentence[i], j] + 1
matrix_trans = matrix.transpose()
co_occurence_matrix = np.add(matrix, matrix_trans)

In [ ]:
#normalize the words by dividing the co-occurence frequency with the total frequency of one word
for i in range(1, V):
    total_tf = np.sum(co_occurence_matrix[:,i])
    if total_tf > 0: #some words do not co-occur with other words, as they are the only words in the sentence (for instance 797)
        for j in range(1,V):
            co_occurence_matrix[j,i] = co_occurence_matrix[j,i]/total_tf

#find cosine similarity to Alice, Dinah and Rabbit
alice_index = tokenizer.word_index['alice']
dinah_index = tokenizer.word_index['dinah']
rabit_index = tokenizer.word_index['rabbit']
#create similarity matrix
sim = cosine_similarity(co_occurence_matrix)
sim_alice_dinah = sim[alice_index, dinah_index]
sim_alice_rabit = sim[alice_index, rabit_index]
sim_dinah_rabit = sim[dinah_index, rabit_index]
print("Cosine similarity Alice - Dinah: %.4f" %sim_alice_dinah)
print("Cosine similarity Alice - Rabbit: %.4f" %sim_alice_rabit)
print("Cosine similarity Dinah - Rabbit: %.4f" %sim_dinah_rabit)

In [ ]:
#find the closest words to Alice
nbrs = nn(n_neighbors=6, algorithm='auto').fit(co_occurence_matrix)
distances, indices = nbrs.kneighbors(co_occurence_matrix)

print(indices[alice_index])
for word, index in tokenizer.word_index.items():    # for name, age in list.items():  (for Python 3.x)
    if index in indices[alice_index]:
        print(word)

In [ ]:
print('The length of the co-occurence matrix is {}'.format(len(co_occurence_matrix)))

Trivially, the closest word to Alice is Alice. However, all other works are not in line with what one would expect. Having read the first lines of Alice in Wonderland, it is for example not clear what the connection is between 'Alice' and 'foot'. A possible explanation is that the dimensionality of the data is high, as the co-occurency of two terms is compared with all (1182-2=)1180. Because of this high dimensionality, the distance difference between the nearest and farthest neighbor is low. To illustrate this, we computed the 100 nearest neighbours to 'Alice' below and compared the longest of these 100 distances with the smallest distance bigger than zero. As can be seen in the output of the code below, the nearest neighbour is only 1.9% closer in terms of distance than the 99th nearest neighbour.

In [ ]:
nbrs_100 = nn(n_neighbors=1000, algorithm='auto').fit(co_occurence_matrix)
distances_1000, indices_1000 = nbrs.kneighbors(co_occurence_matrix)

print(distances[alice_index]/np.average(distances_1000[alice_index]))
print(distances[alice_index]/np.max(distances_1000[alice_index]))
np.max(np.max(distances_1000[alice_index])/distances[alice_index][1:])

In [ ]:
#percentage of cells with zeros
print('The ratio of cells with zeros is {:.4f}'.format
      (np.where(co_occurence_matrix==0)[0].size/co_occurence_matrix.size))

Discussion of the drawback of a term-term co-occurence matrix:
- A co-occurence matrix is very sparse. In this data set, 98,16% of the cells in the word-word co-occurence matrix are zeros. 
- The matrix tends to be very large. For the first 700 lines of the book Alice in Wonderland, we already need a matrix of size 1183x1183. 
- The frequency of occurence can be skewed and non-discriminative. Words like 'the', 'and' and 'to' are not very discriminative, but the impact on the calculation of word similarity of these words is equally big as the impact of more discriminative words. However, this is not necessary a drawback of the use of the word-word co-occurence matrix, as one can adjust for this by using weighting terms.

In [ ]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = [co_occurence_matrix]
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    #Dit werkt wel, maar dan krijg je later weer problemen
#     f.write(" ".join(map(str, list(vectors[0][i,:]))))
    f.write("\n")
f.close()

#reopen your file as follows
co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [3]:
#prepare data for cbow|
def gen(corpus, window_size, V):
    maxlen = window_size * 2
    for sentence in corpus:
        L = len(sentence)
        for index, word in enumerate(sentence):
            context = []
            labels = []
            s = index - window_size
            e = index + window_size + 1
            
            #Add context word (cat is hungry) if  index is 1 then context is cat and hungry
            context.append([sentence[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)
            
            x = sequence.pad_sequences(context, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            #return generator.
            yield (x, y)

In [4]:
dim = 50
cbow_model_50 = Sequential()
cbow_model_50.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_50.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_50.add(Dense(V, activation='softmax'))
#Compile
cbow_model_50.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_50.train_on_batch(x, y)
    print(ite, loss)

0 41686.85704231262
1 39117.07442855835
2 38909.00557476282
3 38815.762184262276
4 38761.799980700016
5 38724.25989371538
6 38676.12575338781
7 38627.986487567425
8 38588.05231449008
9 38562.03263479099


In [5]:
dim = 150
cbow_model_150 = Sequential()
cbow_model_150.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_150.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_150.add(Dense(V, activation='softmax'))
#Compile
cbow_model_150.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_150.train_on_batch(x, y)
    print(ite, loss)

0 41624.23516559601
1 38758.47251689434
2 38354.17740005255
3 38128.833682626486
4 37915.97992122173
5 37717.73056745529
6 37544.31483665854
7 37407.15357390046
8 37298.29355230555
9 37203.6013012426


In [6]:
dim = 300
cbow_model_300 = Sequential()
cbow_model_300.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_300.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_300.add(Dense(V, activation='softmax'))
#Compile
cbow_model_300.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_300.train_on_batch(x, y)
    print(ite, loss)

0 41559.90415096283
1 38492.521023631096
2 37942.09507280588
3 37626.64058974385
4 37343.293199047446
5 37085.93880838156
6 36874.7218532674
7 36709.47696916293
8 36572.27346193092
9 36450.24198226421


In [ ]:
dim = 50
cbow_model_50_extra_layer = Sequential()
cbow_model_50_extra_layer.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_50_extra_layer.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_50_extra_layer.add(Dense(V, activation='selu'))
cbow_model_50_extra_layer.add(Dense(V, activation='softmax'))
#Compile
cbow_model_50_extra_layer.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_50_extra_layer.train_on_batch(x, y)
    print(ite, loss)

In [ ]:
dim = 150
cbow_model_150_extra_layer = Sequential()
cbow_model_150_extra_layer.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_150_extra_layer.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_150_extra_layer.add(Dense(V, activation='selu'))
cbow_model_150_extra_layer.add(Dense(V, activation='softmax'))
#Compile
cbow_model_150_extra_layer.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_150_extra_layer.train_on_batch(x, y)
    print(ite, loss)

In [ ]:
dim = 150
cbow_model_300_extra_layer = Sequential()
cbow_model_300_extra_layer.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow_model_300_extra_layer.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow_model_300_extra_layer.add(Dense(V, activation='selu'))
cbow_model_300_extra_layer.add(Dense(V, activation='softmax'))
#Compile
cbow_model_300_extra_layer.compile(loss='categorical_crossentropy', optimizer='adadelta')
#Train CBOW Model
for ite in range(10):
    loss = 0.
    for x, y in gen(corpus, window_size, V):
        loss += cbow_model_300_extra_layer.train_on_batch(x, y)
    print(ite, loss)

In [ ]:
#Write results
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))
#Write results
cbow_model.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [21]:
#generate data for Skipgram
def generate_data_skipgram(corpus, window_size, V):
    maxlen = window_size*2
    for sentence in corpus:
        L = len(sentence)
        for index, word in enumerate(sentence):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    in_words.append([word])
                    labels.append(sentence[i])
            if in_words != []:
                x = np.array(in_words,dtype=np.int32)
                y = np_utils.to_categorical(labels, V)
                yield (x,y)

In [8]:
dim = 50
#create Skipgram model
skipgram_50 = Sequential()
skipgram_50.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_50.add(Reshape((dim, )))
skipgram_50.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
#define loss function for Skipgram
skipgram_50.compile(loss='categorical_crossentropy', optimizer='adadelta')

#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram(corpus,window_size,V):
        loss += skipgram_50.train_on_batch(x, y)
    print(ite, loss)

0 41271.83740758896
1 39101.409803152084
2 39265.024810791016
3 39357.28198981285
4 39443.99115431309
5 39536.58359324932
6 39641.83396399021
7 39760.61421716213
8 39890.63317453861
9 40026.53615760803


In [9]:
dim = 150
#create Skipgram model
skipgram_150 = Sequential()
skipgram_150.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_150.add(Reshape((dim, )))
skipgram_150.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
#define loss function for Skipgram
skipgram_150.compile(loss='categorical_crossentropy', optimizer='adadelta')

#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram(corpus,window_size,V):
        loss += skipgram_150.train_on_batch(x, y)
    print(ite, loss)

0 41215.39843392372
1 38922.62371468544
2 38984.99028789997
3 39005.47671818733
4 39030.06290924549
5 39068.809734106064
6 39119.5119754076
7 39171.81856572628
8 39215.354902505875
9 39248.874417066574


In [10]:
dim = 300
#create Skipgram model
skipgram_300 = Sequential()
skipgram_300.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_300.add(Reshape((dim, )))
skipgram_300.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
#define loss function for Skipgram
skipgram_300.compile(loss='categorical_crossentropy', optimizer='adadelta')

#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram(corpus,window_size,V):
        loss += skipgram_300.train_on_batch(x, y)
    print(ite, loss)

0 41158.67156767845
1 38737.37768530846
2 38703.30988729
3 38658.188072800636
4 38628.11664748192
5 38608.34821033478
6 38582.74828064442
7 38541.39902639389
8 38489.80666589737
9 38436.1510027647


In [ ]:
dim = 50
#create Skipgram model
skipgram_50_extra_layer = Sequential()
skipgram_50_extra_layer.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_50_extra_layer.add(Reshape((dim, )))
skipgram_50_extra_layer.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='selu'))
skipgram_50_extra_layer.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
#define loss function for Skipgram
skipgram_50_extra_layer.compile(loss='categorical_crossentropy', optimizer='adadelta')

#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram(corpus,window_size,V):
        loss += skipgram_50_extra_layer.train_on_batch(x, y)
    print(ite, loss)

In [ ]:
dim = 150
#create Skipgram model
skipgram_150_extra_layer = Sequential()
skipgram_150_extra_layer.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_150_extra_layer.add(Reshape((dim, )))
skipgram_150_extra_layer.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='selu'))
skipgram_150_extra_layer.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
#define loss function for Skipgram
skipgram_150_extra_layer.compile(loss='categorical_crossentropy', optimizer='adadelta')

#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram(corpus,window_size,V):
        loss += skipgram_150_extra_layer.train_on_batch(x, y)
    print(ite, loss)

In [ ]:
dim = 300
#create Skipgram model
skipgram_300_extra_layer = Sequential()
skipgram_300_extra_layer.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_300_extra_layer.add(Reshape((dim, )))
skipgram_300_extra_layer.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='selu'))
skipgram_300_extra_layer.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
#define loss function for Skipgram
skipgram_300_extra_layer.compile(loss='categorical_crossentropy', optimizer='adadelta')

#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in generate_data_skipgram(corpus,window_size,V):
        loss += skipgram_300_extra_layer.train_on_batch(x, y)
    print(ite, loss)

In [22]:
import operator
def anology_m(model, word_one, word_two, word_three):
    keyvector = model.get_weights()[0]
    index_one = tokenizer.word_index[word_one]
    index_two = tokenizer.word_index[word_two]
    index_three = tokenizer.word_index[word_three]
    result = (keyvector[index_two] - keyvector[index_one]) + keyvector[index_three]
    
    closest_words = {}
    closest_word = ""
    min_score = -1
    for word, index in tokenizer.word_index.items():
        if(index not in [index_one, index_two, index_three]):
            sim_score = cosine_similarity([keyvector[index], result])[0][1]
            closest_words[word] = sim_score
            if(sim_score > min_score):
                min_score = sim_score
                closest_word = word
    sorted_x = sorted(closest_words.items(), key=operator.itemgetter(1), reverse=True)
    return dict(sorted_x[:10])

In [30]:
#Happy not in dataset hence we catch keyerror
def test_acc(model):
    hit_1 = 0
    hit_5 = 0
    hit_10 = 0
    file = open('analogy_alice.txt')
    lines = file.readlines()
    for line in lines:
        try:
            tokens = line.strip().split(' ')
            result = anology_m(model, tokens[0], tokens[1], tokens[2])
            if(tokens[3] == result[0].keys()):
                hit_1 += 1
            elif(tokens[3] in result[:5].keys()):
                hit_5 += 1
            elif(tokens[3] in result.keys()):
                hit_10 += 1
        except KeyError:
            pass
    return hit_1, hit_5, hit_10

In [31]:
#cbow without extra layer results
res_cbow_50_1, res_cbow_50_2, res_cbow_50_3 = test_acc(cbow_model_50)
res_cbow_150_1, res_cbow_150_2, res_cbow_150_3 = test_acc(cbow_model_150)
res_cbow_300_1, res_cbow_300_2, res_cbow_300_3 = test_acc(cbow_model_300)

In [32]:
print(res_cbow_300_3)

0


In [33]:
#skipgram without extra layer results
res_skipgram_50_1, res_skipgram_50_2, res_skipgram_50_3 = test_acc(skipgram_50)
res_skipgram_150_1, res_skipgram_150_2, res_skipgram_150_3 = test_acc(skipgram_150)
res_skipgram_300_1, res_skipgram_300_2, res_skipgram_300_3 = test_acc(skipgram_300)

In [34]:
print(res_skipgram_300_3)

0


In [ ]:
#cbow wextra layer results
res_cbow_el_50_1, res_cbow_el_50_2, res_cbow_el_50_3 = test_acc(cbow_model_50_extra_layer)
res_cbow_el_150_1, res_cbow_el_150_2, res_cbow_el_150_3 = test_acc(cbow_model_150_extra_layer)
res_cbow_el_300_1, res_cbow_el_300_2, res_cbow_el_300_3 = test_acc(cbow_model_300_extra_layer)

In [ ]:
#skipgram with extra layer results
res_skipgram_el_50_1, res_skipgram_el_50_2, res_skipgram_el_50_3 = test_acc(skipgram_50_extra_layer)
res_skipgram_el_150_1, res_skipgram_el_150_2, res_skipgram_el_150_3 = test_acc(skipgram_150_extra_layer)
res_skipgram_el_300_1, res_skipgram_el_300_2, res_skipgram_el_300_3 = test_acc(skipgram_300_extra_layer)

In [36]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [39]:
def tsne(model):
    labels = []
    tokens = []
    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)
    
    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
    
    plt.figure(figsize=(14,14))
    for i in range(len(x)):
        plt.scatter(x[i], y[i])
        plt.annotate(labels[i],
                    xy=(x[i], y[i]),
                    xytext=(5,2),
                    textcoords='offset points',
                    ha='right',
                    va='bottom')
        plt.show()

In [38]:
#load pretrained word embeddings of word2vec

path_word2vec = "C:/Users/Niek/Documents/school/RedSystems/data/GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path_word2vec, binary=True)

In [42]:
print(word2vec.vocab)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings